In [ ]:
#!{sys.executable} -m pip install Pillow

from pathlib import Path
from PIL import Image
import easyocr
import numpy as np
from sklearn.cluster import DBSCAN
import numpy as np
import os
from pathlib import Path
from paddleocr import PaddleOCR
from PIL import Image, ImageDraw, ImageFont



In [3]:

print(os.getcwd())
input_folder = Path(r"input\Drawing Saikyou Mangaka wa Oekaki Skill de Isekai Musou Suru!\Vol. 1 Ch. 1")

image_files = sorted(input_folder.glob("*.jpg"))

images = [Image.open(f) for f in image_files]
# Initialize EasyOCR reader
reader = easyocr.Reader(['en'], gpu=False)



Using CPU. Note: This module is much faster with a GPU.


d:\Documents\Jupyter\MT


In [ ]:
# Allowlist for uppercase manga text and common punctuation
allow_chars = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!?.,-…'\""
ocr_results = []
# EPS variable sets box size.  It should probably be 80 to group text boxes.  At 40 each word is its own box.
eps_var = 40

for img in images:
    # Apply allowlist during OCR, with adjusted width_ths to reduce box merging
    results = reader.readtext(img, allowlist=allow_chars, width_ths=0.005)
    ocr_results.append(results)

def group_bubbles_2d(results, eps=eps_var, min_samples=1):
    """
    Group OCR results into bubbles using 2D clustering.
    
    eps: max distance in pixels to consider points in same cluster
    min_samples: min number of words to form a cluster
    """
    if not results:
        return []

    # Compute center coordinates for each word
    coords = []
    for bbox, text, conf in results:
        x_center = np.mean([p[0] for p in bbox])
        y_center = np.mean([p[1] for p in bbox])
        coords.append([x_center, y_center]) 
#        print(bbox)
#        print(" . ")
#        print(text)
    coords = np.array(coords)
    
    # DBSCAN clustering in 2D
    clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(coords)
    labels = clustering.labels_
    
    bubbles = []
    for cluster_id in set(labels):
        if cluster_id == -1:
            # Noise, treat as its own bubble
            cluster_indices = [i for i, label in enumerate(labels) if label == cluster_id]
            cluster_words = sorted(
                [results[i] for i in cluster_indices],
                key=lambda x: (np.mean([p[0] for p in x[0]]), np.mean([p[1] for p in x[0]]))
            )
            bubble_text = " ".join([w[1] for w in cluster_words])
            bubbles.append(bubble_text)
        else:
            cluster_indices = [i for i, label in enumerate(labels) if label == cluster_id]
            # Sort words in cluster top-to-bottom, then left-to-right (using centers for stability)
            cluster_words = sorted(
                [results[i] for i in cluster_indices],
                key=lambda x: (np.mean([p[1] for p in x[0]]), np.mean([p[0] for p in x[0]]))
            )
            bubble_text = " ".join([w[1] for w in cluster_words])
            bubbles.append(bubble_text)
    return bubbles

# Apply clustering and print
for i, image_results in enumerate(ocr_results):
    bubbles = group_bubbles_2d(image_results, eps=eps_var)
    print(f"\n--- Image {i+1} ---")
    for b in bubbles:
        print(b)

C:\Users\John\AppData\Roaming\Python\Python313\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



--- Image 1 ---
I'C
ONLY
WHAT STUPID.
I
NATURAL THATL PROTECT YOU
HOW I
CAN
THINGS
ARE SAYING.
YOU
NO SNACKS
YOU
ARE
MY FAMILY.
STAY
QUICO45-
STUPID
OF
HEREI!
FOR
SUCH
A
LONG
IM AKIRA'S GUARDIAN
TIME..

--- Image 2 ---
I
SHOULD
GO
BACK,.-
AND
APOLOGIZE
TO
ARIA..
THAVESAID TOOMUCH
SOMEONE
LIKE
ARIA.JJ!
IT'S
THE TIME. IHAVESEEN LIKE
FIRST
IS
BETTER GONE!!
HER
THAT


In [1]:
from pathlib import Path
from PIL import Image
import easyocr
import numpy as np
from sklearn.cluster import DBSCAN
import numpy as np
import os
from pathlib import Path
from paddleocr import PaddleOCR
from PIL import Image, ImageDraw, ImageFont


# --------------------------
# CONFIG
# --------------------------
input_folder = Path(r"input\Drawing Saikyou Mangaka wa Oekaki Skill de Isekai Musou Suru!\Vol. 1 Ch. 1")
output_folder = Path("output")
output_folder.mkdir(exist_ok=True)

allow_chars = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!?.,-…'\""
replacements = {"JUMP": "LEAP"}  # Dictionary of word replacements
font_path = "arial.ttf"  # Use a comic font if you have one
font_size = 20  # Adjust size to fit speech bubbles

# --------------------------
# LOAD IMAGES
# --------------------------
image_files = sorted(input_folder.glob("*.jpg"))
images = [Image.open(f) for f in image_files]

# --------------------------
# INITIALIZE OCR
# --------------------------
reader = easyocr.Reader(['en'], gpu=False)
ocr_results_all = []

# --------------------------
# OCR ALL IMAGES
# --------------------------
for img in images:
    results = reader.readtext(np.array(img), allowlist=allow_chars, width_ths=0.005)
    ocr_results_all.append(results)

# --------------------------
# REPLACE WORDS
# --------------------------
def replace_words(results, replacements):
    """
    Replace words in OCR results according to a dictionary.
    """
    new_results = []
    for bbox, text, conf in results:
        new_text = " ".join([replacements.get(word.upper(), word) for word in text.split()])
        new_results.append((bbox, new_text, conf))
    return new_results

ocr_results_modified = [replace_words(r, replacements) for r in ocr_results_all]

# --------------------------
# DRAW TEXT BACK ON IMAGE
# --------------------------
def draw_text_on_image(img, results, font_path=font_path, font_size=font_size):
    """
    Draw modified text on a PIL image.
    """
    pil_img = img.copy()
    draw = ImageDraw.Draw(pil_img)
    font = ImageFont.truetype(font_path, font_size)

    for bbox, text, conf in results:
        # Calculate approximate center of the bbox
        x_center = int(np.mean([p[0] for p in bbox]))
        y_center = int(np.mean([p[1] for p in bbox]))
        draw.text((x_center, y_center), text, fill="black", font=font, anchor="mm")

    return pil_img

# --------------------------
# PROCESS AND SAVE IMAGES
# --------------------------
for idx, img in enumerate(images):
    modified_img = draw_text_on_image(img, ocr_results_modified[idx])
    output_path = output_folder / f"page_{idx+1:03d}.png"
    modified_img.save(output_path)
    print(f"Saved modified page: {output_path}")


Using CPU. Note: This module is much faster with a GPU.
C:\Users\John\AppData\Roaming\Python\Python313\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Saved modified page: output\page_001.png
Saved modified page: output\page_002.png
